In [1]:
import os
import pandas as pd

input_folder ="D:\西安22运行数据\运行 22\行驶在西安内的文件，里面包含了删除行驶在西安外的数据\需要预处理的"
output_folder = "D:\西安22运行数据\运行 22\预处理"

# 获取文件夹中所有文件的大小并按大小排序
files = [(f, os.path.getsize(os.path.join(input_folder, f))) for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
files.sort(key=lambda x: x[1])

# 读取、处理和保存文件
for file, size in files:
    data = pd.read_csv(os.path.join(input_folder, file))
    
    # 在这里对data运行你的代码
    
    
    data = pd.DataFrame(data)
    print('data的长度:',len(data))
    #对于定位时间这一列，删除不是2022年的数据

    data['定位年份']=data['定位时间'].apply(lambda r:r.split('-')[0])    
    i = 0
    while i <len(data):
        data.iloc[i, 18]=int(data.iloc[i,18])
        i += 1
    print('转换完成')
    data=data[data['定位年份']==2022]
    data.drop("定位年份", axis=1, inplace=True)
    data.reset_index(drop=True, inplace=True)
    print("删除时间不是2022年的之后的数据：",len(data))

    #删除定位时间重复的数据

    data = data.drop_duplicates(subset=['定位时间'], keep='first')
    data.reset_index(drop=True, inplace=True)
    print("删除定位时间重复后的数据数量：",len(data))

    #去除车速异常数据：车速大于120km/h，即每秒速度高于33m/s
    data = data[data['车速'] <=120]
    data.reset_index(drop=True, inplace=True)
    print("删除车速异常数据",len(data))


    #重新定义了时间戳
    data['定位时间'] = pd.to_datetime(data['定位时间'])
    # 定义函数来处理时间戳
    def define_timestamp(group_data):
        group_data['时间戳'] = (group_data['定位时间'] - group_data['定位时间'].iloc[0]).dt.total_seconds() + 1
        return group_data

    # 根据行程id和子行程id对数据进行分组，并应用时间戳处理函数
    data = data.groupby(['行程id', '子行程id'], group_keys=False).apply(define_timestamp).reset_index(drop=True)

    print("重新定义了时间戳")


    # 将车速从km/h转换为m/s
    data['车速'] = data['车速'] * (1000 / 3600)
    print("将车速从km/h转换为m/s")


    #定义加速度，单位m/s^2
    i=1
    while i<len(data)-1:

        if (data.at[i,"子行程id"]==data.at[i+1,"子行程id"])and (data.at[i,"行程id"]==data.at[i+1,"行程id"]):
            tf = data.at[i+1,'时间戳']-data.at[i,'时间戳']   #时间差
            sf = data.at[i+1,'车速']-data.at[i,'车速']    #速度差
            data.loc[i, '加速度(m/s^2)']=sf/tf
        else :
            data.loc[i, '加速度(m/s^2)'] = data.loc[i-1,'加速度(m/s^2)']
            data.loc[i+1, '加速度(m/s^2)'] =0
            i=i+1
        i+=1    
    data.loc[0, '加速度(m/s^2)'] = data.loc[1, '加速度(m/s^2)']  
    data.loc[len(data)-1, '加速度(m/s^2)'] = data.loc[len(data)-2, '加速度(m/s^2)']
    print("定义了加速度",len(data))

    #去除经度和纬度不变化但车速的数据

    i = 0
    while i < len(data)-1:
        if ((data.at[i,"子行程id"]==data.at[i+1,"子行程id"])and (data.at[i,"行程id"]==data.at[i+1,"行程id"])) and ((data.iloc[i, data.columns.get_loc("经度")] - data.iloc[i+1, data.columns.get_loc("经度")] == 0) and (data.iloc[i, data.columns.get_loc("纬度")] - data.iloc[i+1, data.columns.get_loc("纬度")] == 0) and (data.iloc[i, data.columns.get_loc("车速")] - data.iloc[i+1, data.columns.get_loc("车速")] != 0)):
            data = data.drop(i, axis=0).reset_index(drop=True)
        else:
            i += 1
    print("除经度和纬度不变化但车速的数据")

    #去除加速度异常的数据
    data = data[(data['加速度(m/s^2)'] >= -4) & (data['加速度(m/s^2)'] <= 4)]
    data.reset_index(drop=True, inplace=True)
    print("去除了加速度异常的数据",len(data))


    #拉格朗日插值，将数据变成1s间隔的

    def lagrange_interpolation_sorted(data):
        interpolated_data = []

        for (trip_id, sub_trip_id), group in data.groupby(['行程id', '子行程id']):
            prev_row = None
            for idx, row in group.iterrows():
                if prev_row is not None and row['时间戳'] - prev_row['时间戳'] <= 5:
                    time_diff = row['时间戳'] - prev_row['时间戳']
                    value_diff = row['定位时间'] - prev_row['定位时间']
                    interval = 1  # 设定插值后数据的定位时间间隔为1s
                    steps = int((row['时间戳'] - prev_row['时间戳']) / interval)
                    for i in range(1, steps):
                        interpolated_row = prev_row.copy()
                        interpolated_row['定位时间'] = prev_row['定位时间'] + (value_diff / time_diff) * i
                        interpolated_row['时间戳'] = prev_row['时间戳'] + i
                        interpolated_data.append(interpolated_row)

                prev_row = row

        # 将插值后的数据添加到原始数据中
        if interpolated_data:
            interpolated_data = pd.DataFrame(interpolated_data)
            data = pd.concat([data, interpolated_data], ignore_index=True)

        # 按定位时间排序并重置索引
        data.sort_values(by='定位时间', inplace=True)
        data.reset_index(drop=True, inplace=True)

        return data
    # 使用函数处理数据
    data = lagrange_interpolation_sorted(data)
    print("拉格朗日插值",len(data))



    #长期（超过 50s）低速行驶的数据，车速小于10km/h，将车速变为0
    # 将车速小于10km/h且持续时间超过50秒的段落车速赋值为0的函数
    def process_speed(group):
        speed_limit = 2.77777777777  # 设置速度限制
        duration_threshold = 50  # 设置持续时间阈值（单位：秒）

        # 找出符合条件的索引
        indices = (group['车速'] < speed_limit).astype(int).diff().ne(0).cumsum()
        subgroups = group.groupby(indices)

        for _, subgroup in subgroups:
            if (subgroup['车速'] < speed_limit).all() and len(subgroup) >= duration_threshold:  # 添加额外的条件检查
                group.loc[subgroup.index, '车速'] = 0

        return group
    data = data.groupby(['行程id', '子行程id'], group_keys=False).apply(process_speed)
    print("将长期低速行驶的数据改为怠速了",len(data))

    #怠速行驶超过180s的数据截断。怠速：车速为0但是转速不为0
    data['怠速状态'] = data.apply(lambda row: 'T' if row['车速'] == 0 and row['转数'] != 0 else 'F', axis=1)

    i = 0
    n = 0
    lock = 0
    while i < len(data)-1:
        if data.iat[i,1]==data.iat[i+1,1]:
            if lock == 0 and data.iloc[i, 19]=='T': # 根据索引
                s = i
                #print(s)
                lock = 1
            if data.iloc[i, 19]=='T':
                count = (data.iloc[i,11] - data.iloc[s,11])
                if count > 180:
                    data.drop(i+n, inplace=True) #根据行名
                    n += 1
                else:
                    i += 1
            else:
                lock = 0
                i += 1 
        else:
            i += 1
    data.reset_index(drop=True, inplace=True)
    print("截断过长时间怠速的数据数量：",len(data))
    print("---------------------------------------------------------------")
    output_file_path = os.path.join(output_folder, file)
    data.to_csv(output_file_path, index=False)


data的长度: 435662
转换完成
删除时间不是2022年的之后的数据： 435662
删除定位时间重复后的数据数量： 419836
删除车速异常数据 419836
重新定义了时间戳
将车速从km/h转换为m/s
定义了加速度 419836
除经度和纬度不变化但车速的数据
去除了加速度异常的数据 416507
拉格朗日插值 595034
将长期低速行驶的数据改为怠速了 595034
截断过长时间怠速的数据数量： 510678
---------------------------------------------------------------
data的长度: 439672
转换完成
删除时间不是2022年的之后的数据： 439672
删除定位时间重复后的数据数量： 410469
删除车速异常数据 408961
重新定义了时间戳
将车速从km/h转换为m/s
定义了加速度 408961
除经度和纬度不变化但车速的数据
去除了加速度异常的数据 406036
拉格朗日插值 634284
将长期低速行驶的数据改为怠速了 634284
截断过长时间怠速的数据数量： 601713
---------------------------------------------------------------
data的长度: 444157
转换完成
删除时间不是2022年的之后的数据： 444151
删除定位时间重复后的数据数量： 398272
删除车速异常数据 397864
重新定义了时间戳
将车速从km/h转换为m/s
定义了加速度 397864
除经度和纬度不变化但车速的数据
去除了加速度异常的数据 392576
拉格朗日插值 422854
将长期低速行驶的数据改为怠速了 422854
截断过长时间怠速的数据数量： 403245
---------------------------------------------------------------
data的长度: 441839
转换完成
删除时间不是2022年的之后的数据： 441839
删除定位时间重复后的数据数量： 400883
删除车速异常数据 400883
重新定义了时间戳
将车速从km/h转换为m/s
定义了加速度 400883
除经度和纬度不变化但车速的数据
去除了加速度异常的数据 39